In [1]:
import pandas as pd
from transformers import pipeline
import time
from utils import examples, intents, entities
from language_model import llm_generate
import ast

tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\first\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\first\.cache\huggingface\hub\models--Qwen--Qwen2.5-0.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

LLM Inference accuracy(Performance), time(Latency)

In [33]:
from utils import examples, intents, entities
llm_out = []
for user_input, true_intent, true_entity in zip(examples, intents, entities):
        llm_output = llm_generate(user_input)
        llm_out.append(ast.literal_eval(llm_output))
        print(f"User Input: {user_input}")
        print(f"LLM Output: {llm_output}")
        print(f"True Intent: {true_intent}, True Entities: {true_entity}")
        print("-" * 50) 


User Input: Open YouTube
LLM Output: {"intent": "open_app", "entities": [{"type": "app_name", "value": "YouTube"}]}
True Intent: open_app, True Entities: [{'type': 'app_name', 'value': 'YouTube'}]
--------------------------------------------------
User Input: Search for cooking videos on YouTube
LLM Output: {"intent": "search", "entities": [{"type": "search_query", "value": "cooking videos"}, {"type": "app_name", "value": "YouTube"}]}
True Intent: search, True Entities: [{'type': 'search_query', 'value': 'cooking videos'}, {'type': 'app_name', 'value': 'YouTube'}]
--------------------------------------------------
User Input: Turn up the volume
LLM Output: {"intent": "settings", "entities": [{"type": "settings_action", "value": "volume_up"}]}
True Intent: settings, True Entities: [{'type': 'settings_action', 'value': 'volume_up'}]
--------------------------------------------------
User Input: Find action movies
LLM Output: {"intent": "play_media", "entities": [{"type": "content_type", 

In [38]:
# calculate accuracy
pred_intents = [output['intent'] for output in llm_out]
pred_intents = [intent if intent != "play_media" else "search" for intent in pred_intents]
correct = sum(p == t for p, t in zip(pred_intents, intents))
accuracy = correct / len(intents)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 87.50%


In [39]:
# Calculate Time 
llm_latencies = []
for example in examples[:4]:
    start_time = time.time()
    for _ in range(5):  # Run multiple times for averaging
        llm_generate(example)
    end_time = time.time()
    avg_latency = (end_time - start_time) / 5
    llm_latencies.append(avg_latency)

for example, latency in zip(examples[:4], llm_latencies):
    print(f"Example: '{example}' - Average Latency: {latency:.4f} seconds")

Example: 'Open YouTube' - Average Latency: 43.2046 seconds
Example: 'Search for cooking videos on YouTube' - Average Latency: 41.3157 seconds
Example: 'Turn up the volume' - Average Latency: 36.4100 seconds
Example: 'Find action movies' - Average Latency: 37.0170 seconds


Classifier

In [40]:
classifier = pipeline(
    "zero-shot-classification",
    model="distilbert/distilbert-base-multilingual-cased",  # 134M parameters 
)
labels = ["open_app", "search", "settings", "None"]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Inference: accuracy(Performance), Time(Latency)

In [41]:
result = classifier(examples, candidate_labels=labels)

pred_intents = []
for true_intent, res in zip(intents, result):
    pred_intents.append(res['labels'][0])
    pred_label = res['labels'][0]
    pred_score = res['scores'][0]

    print(f"True Intent: {true_intent}")
    print(f"Predicted Intent: {pred_label} with score {pred_score:.4f}")
    print()

True Intent: open_app
Predicted Intent: search with score 0.2518

True Intent: search
Predicted Intent: open_app with score 0.2514

True Intent: settings
Predicted Intent: open_app with score 0.2526

True Intent: search
Predicted Intent: None with score 0.2520

True Intent: None
Predicted Intent: open_app with score 0.2515

True Intent: settings
Predicted Intent: open_app with score 0.2521

True Intent: search
Predicted Intent: open_app with score 0.2513

True Intent: open_app
Predicted Intent: open_app with score 0.2509



In [42]:
# Calculate accuracy
correct = sum(t == p for t, p in zip(intents, pred_intents))
accuracy = correct / len(intents)
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 12.50%


In [43]:
# Calculate Time 
cls_latencies = []
for example in examples:
    start_time = time.time()
    for _ in range(5):  # Run multiple times for averaging
        classifier(example, candidate_labels=labels)
    end_time = time.time()
    avg_latency = (end_time - start_time) / 5
    cls_latencies.append(avg_latency)

for example, latency in zip(examples, cls_latencies):
    print(f"Example: '{example}' - Average Latency: {latency:.4f} seconds")


Example: 'Open YouTube' - Average Latency: 0.1311 seconds
Example: 'Search for cooking videos on YouTube' - Average Latency: 0.1606 seconds
Example: 'Turn up the volume' - Average Latency: 0.1462 seconds
Example: 'Find action movies' - Average Latency: 0.1296 seconds
Example: 'Good to see you again!' - Average Latency: 0.1404 seconds
Example: 'Mute the TV' - Average Latency: 0.1472 seconds
Example: 'I want to watch Ronaldo Goals' - Average Latency: 0.1422 seconds
Example: 'Launch Netflix' - Average Latency: 0.1344 seconds


LLM, Classifier speedup 

In [47]:
speedup_factors = [llm / cls for cls, llm in zip(cls_latencies, llm_latencies)]
for speedup in speedup_factors: 
    print(f"Speedup Factor: {speedup:.2f}x")

Speedup Factor: 329.55x
Speedup Factor: 257.26x
Speedup Factor: 249.04x
Speedup Factor: 285.71x


Intent Data

In [6]:
import pandas as pd
eng_df = pd.read_csv("data/english_intent_dataset.csv")
ara_df = pd.read_csv("data/arabic_intents_clean_balanced.csv")

In [7]:
eng_df.head()

,text,label
0,launch YouTube and perform a search for cat vi...,open_app_and_search
1,adjust sharpness,settings
2,search travel documentaries,search
3,raise the brightness,settings
4,Disney Plus service go and search animations,open_app_and_search


In [8]:
ara_df.head()

,text,label
0,فين ألاقي أسعار السيارات,Search
1,دلني على وصفات طبخ,Search
2,فعل الموقع,Settings
3,أريد معرفة معلومات عن كورسات برمجة,Search
4,اذهب إلى إعدادات الشاشة,Settings


In [9]:
eng_df.label.value_counts()

label
search                 238
settings               231
open_app               224
out_of_scope           220
open_app_and_search    195
Name: count, dtype: int64

In [10]:
ara_df.label.value_counts()

label
Search                 250
Open_app               250
Out_of_scope           250
Open_app_and_search    250
Settings               249
Name: count, dtype: int64

Merge Datasets

In [14]:
df = pd.concat([eng_df, ara_df], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(20)

,text,label
0,مرحبا,Out_of_scope
1,open HBO Max and look for Game of Thrones,open_app_and_search
2,مرحبا,Out_of_scope
3,أريد معرفة معلومات عن أسعار السيارات,Search
4,look for fantasy series,search
5,show me survival shows,search
6,search zombie shows,search
7,أريد معرفة معلومات عن تمارين رياضية,Search
8,افتح تطبيق TikTok,Open_app
9,look for cooking shows,search


In [ ]:
df.label.value_counts()

label
Out_of_scope           250
Search                 250
Open_app_and_search    250
Open_app               250
Settings               249
search                 238
settings               231
open_app               224
out_of_scope           220
open_app_and_search    195
Name: count, dtype: int64

In [17]:
def uncapitalize_first_letter(text):
    if isinstance(text, str) and text:
        return text[0].lower() + text[1:]
    return text

df["label"] = df["label"].apply(uncapitalize_first_letter)

In [16]:
df.label.value_counts() # 488, 480, 474, 470, 445

label
out_of_scope           530
search                 490
settings               480
open_app               474
open_app_and_search    460
Name: count, dtype: int64

In [17]:
df.to_csv("data\multilingual_intent_dataset.csv", index=False)

Trained Classifier

In [14]:
model_path = "mohamedgomaaa/intent-classifier-multilingual"
model1 = pipeline(
    "text-classification",
    model=model_path,
    top_k=None)

ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /api/models/mohamedgomaaa/intent-classifier-multilingual (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026C85FF3550>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 854e407f-d0b9-42d1-be5c-b8ca05321650)')

In [2]:
def classifier_pred(model, texts):
    outs = model(texts)
    pred_label = [out[0]['label'] for out in outs]
    return pred_label

In [2]:
def cal_accuracy(model, texts, labels):
    pred_label = classifier_pred(model, texts)
    correct = sum(p == t for p, t in zip(pred_label, labels))
    accuracy = correct / len(labels)
    return accuracy

Evaluate Training Data

In [6]:
df = pd.read_csv("data/multilingual_intent_dataset.csv")
texts = df['text'].tolist()
labels = df['label'].tolist()

In [8]:
cal_accuracy(model1, texts, labels)

0.9819227608874281

Evaluate Testing Data

In [9]:
df = pd.read_csv("data/intent_test_data.csv")
texts = df['text'].tolist()
labels = df['label'].tolist()

In [10]:
cal_accuracy(model1, texts, labels)

0.92

Evaluate each class

In [11]:
classes = ['search', 'open_app', 'open_app_and_search', 'settings', 'out_of_scope']
acc_class = []
def get_class(label):
    text_class = df[df['label'] == label]['text'].tolist()
    label_class = df[df['label'] == label]['label'].tolist()
    return text_class, label_class

In [13]:
for clss in classes:
    text, label = get_class(clss)
    acc_class.append((clss, cal_accuracy(model1, text, label)))
acc_class

[('search', 0.9545454545454546),
 ('open_app', 0.95),
 ('open_app_and_search', 1.0),
 ('settings', 1.0),
 ('out_of_scope', 0.6666666666666666)]

In [14]:
texts, labels = get_class("out_of_scope")
outs = model1(texts)
pred_labels = [out[0]['label'] for out in outs]
for label, pred_label, text in zip(labels, pred_labels, texts):
    if label != pred_label:
        print(text, pred_label)

set alarm for 7am settings
order food delivery settings
calculate tip for 50 dollars settings
turn on the lights settings
play a game settings
translate this to German settings


Trained Classifier v2

In [3]:
model_path = "mohamedgomaaa/intent-classifier-multilingual-v2"
model2 = pipeline(
    "text-classification",
    model=model_path,
    top_k=None)

Device set to use cpu


Evaluate Training Data

In [16]:
df = pd.read_csv("data/multilingual_intent_dataset.csv")
texts = df['text'].tolist()
labels = df['label'].tolist()

In [17]:
cal_accuracy(model2, texts, labels)

0.9991783073130649

Evaluate Testing Data

In [18]:
df = pd.read_csv("data/intent_test_data.csv")
texts = df['text'].tolist()
labels = df['label'].tolist()

In [20]:
cal_accuracy(model2, texts, labels)

0.99

Evaluate on large test dataset

In [21]:
df = pd.read_csv("data/test_dataset_200.csv")
texts = df['text'].tolist()
labels = df['label'].tolist()

In [22]:
cal_accuracy(model1, texts, labels)

0.8838383838383839

In [23]:
cal_accuracy(model2, texts, labels)

0.9545454545454546

In [25]:
acc_class = []
for clss in classes:
    text, label = get_class(clss)
    acc_class.append((clss, cal_accuracy(model2, text, label)))
acc_class

[('search', 0.975),
 ('open_app', 0.95),
 ('open_app_and_search', 0.975),
 ('settings', 1.0),
 ('out_of_scope', 0.8717948717948718)]

In [26]:
texts, labels = get_class("out_of_scope")
outs = model2(texts)
pred_labels = [out[0]['label'] for out in outs]
for label, pred_label, text in zip(labels, pred_labels, texts):
    if label != pred_label:
        print(text, pred_label)

start a game of trivia open_app
convert this to Spanish settings
find parking near me search
compose an email settings
set timer for 15 minutes settings


LLM Testing

In [5]:
import json
with open("data/llm_test.json", "r") as f:
    examples = json.load(f)

examples[0]

{'idx': 1,
 'example': 'find superhero movies for tonight',
 'classifier_out': 'search',
 'llm_out': '{"search_query":"superhero movies"}'}

Classifier + LLM

In [6]:
import json
outputs = []   
for example in  examples:
    t1 = time.time()
    classifier_out = classifier_pred(model2, example['example'])[0]
    t2 = time.time()
    llm_out = llm_generate(example['example'], classifier=classifier_out)
    t3 = time.time()
    outputs.append({"idx": example['idx'], "example": example['example'], "classifier_out": classifier_out, "llm_out": llm_out, "classifier_time": t2 - t1, "llm_time": t3 - t2, "total_time": t3 - t1})


with open("output/output_qwen_classifier.json", "w", encoding="utf-8") as f:
    json.dump(outputs, f, indent=2, ensure_ascii=False)     


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


LLM Llama Evaluation (Accuracy, Latency)

In [8]:
from utils import evaluate_perf_latency

In [12]:
evaluate_perf_latency(results_file="output\output_llama_classifier.json", expected_file="data\llm_test.json")


SEARCH
  Accuracy: 5/11 = 45.45%
  Avg Time: 16.48s

OPEN_APP
  Accuracy: 2/10 = 20.00%
  Avg Time: 13.40s

OPEN_APP_AND_SEARCH
  Accuracy: 8/10 = 80.00%
  Avg Time: 15.72s

SETTINGS
  Accuracy: 7/10 = 70.00%
  Avg Time: 16.08s

OUT_OF_SCOPE
  Accuracy: 0/9 = 0.00%
  Avg Time: 17.17s

TOTAL
  Accuracy: 22/50 = 44.00%
  Avg Time: 15.75s


LLM Qwen Evaluation (Accuracy, Latency)

In [9]:
evaluate_perf_latency(results_file="output\output_qwen_classifier.json", expected_file="data\llm_test.json")


SEARCH
  Accuracy: 6/11 = 54.55%
  Avg Time: 6.46s

OPEN_APP
  Accuracy: 7/10 = 70.00%
  Avg Time: 5.54s

OPEN_APP_AND_SEARCH
  Accuracy: 10/10 = 100.00%
  Avg Time: 6.15s

SETTINGS
  Accuracy: 7/10 = 70.00%
  Avg Time: 5.75s

OUT_OF_SCOPE
  Accuracy: 0/9 = 0.00%
  Avg Time: 6.47s

TOTAL
  Accuracy: 30/50 = 60.00%
  Avg Time: 6.07s


Classifier + Regex + LLM

In [6]:
from utils import extract_app_name, extract_settings_action

In [8]:
import json
outputs = []   
for example in examples:
    t1 = time.time()
    classifier_out = classifier_pred(model2, example['example'])[0]
    t2 = time.time()
    if classifier_out == "open_app":
       entities = extract_app_name(example['example'])
    elif classifier_out == "settings":
       entities = extract_settings_action(example['example'])
    elif classifier_out == "out_of_scope":
       entities = "I can help with searching for content, opening applications, and changing settings."
    elif classifier_out == "open_app_and_search":
        entity_app = json.loads(extract_app_name(example['example']))
        search_query = llm_generate(example['example'], classifier="search")
        if search_query == "null":
            search_query = {"search_query": "null"}
        else:
            search_query = json.loads(search_query)
        entities = json.dumps({"app_name": entity_app["app_name"], "search_query": search_query['search_query']}, separators=(',', ':'))
    elif classifier_out == "search":
         entities = llm_generate(example['example'], classifier="search")
         
    t3 = time.time()
    outputs.append({"idx": example['idx'], "example": example['example'], "classifier_out": classifier_out, "llm_out": entities, "classifier_time": t2 - t1, "llm_time": t3 - t2, "total_time": t3 - t1})


with open("output/output_llama_regex_classifier.json", "w", encoding="utf-8") as f:
    json.dump(outputs, f, indent=2, ensure_ascii=False)  

In [9]:
evaluate_perf_latency(results_file="output\output_llama_regex_classifier.json", expected_file="data\llm_test.json")


SEARCH
  Accuracy: 5/11 = 45.45%
  Avg Time: 16.07s

OPEN_APP
  Accuracy: 9/10 = 90.00%
  Avg Time: 0.03s

OPEN_APP_AND_SEARCH
  Accuracy: 1/10 = 10.00%
  Avg Time: 15.76s

SETTINGS
  Accuracy: 6/10 = 60.00%
  Avg Time: 0.03s

OUT_OF_SCOPE
  Accuracy: 9/9 = 100.00%
  Avg Time: 0.03s

TOTAL
  Accuracy: 30/50 = 60.00%
  Avg Time: 6.71s


ONNX

In [ ]:
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer, pipeline

# 1. Load quantized ONNX model
model = ORTModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    export=True,
    provider="CUDAExecutionProvider",  # or CPUExecutionProvider
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# 2. Create pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# 3. Test speed
import time
start = time.time()
output = llm_generate("Open Netflix")
print(f"Time: {time.time() - start}s")